# Design a Summary/Translation Prompt for DialogSum

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
!pip install --upgrade fsspec datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


## Summary with Few-Shot learning

In [4]:
# Select three sample dialogues with their summaries
few_shot_examples = [
    "Summarise: '#Person1#: Excuse me, did you see a set of keys? #Person2#: What kind of keys? #Person1#: Five keys and a small foot ornament. #Person2#: What a shame! I didn't see them. #Person1#: Well, can you help me look for it? That's my first time here. #Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys. #Person1#: It's very kind of you. #Person2#: It's not a big deal. Hey, I found them. #Person1#: Oh, thank God! I don't know how to thank you, guys. #Person2#: You're welcome.' Summary: '#Person1#'s looking for a set of keys and asks for #Person2#'s help to find them.'",
    "Summarise: '#Person1#: This is a good basic computer package. It's got a good CPU, 256 megabytes of RAM, and a DVD player. #Person2#: Does it come with a modem? #Person1#: Yes, it has a built-in modem. You just plug a phone line into the back of the computer. #Person2#: How about the monitor? #Person1#: A 15 - inch monitor is included in the deal. If you want, you can switch it for a 17 - inch monitor, for a little more money. #Person2#: That's okay. A 15 - inch is good enough. All right, I'll take it.' Summary: '#Person1# shows a basic computer package to #Person2#. #Person2# thinks it's good and will take it.'",
     "Summarise: '#Person1#: Hello Mrs. Parker, how have you been? #Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines. #Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots. #Person2#: What about Rubella and Mumps? #Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest. #Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago! #Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.' Summary: 'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'"
    ]

# New prompt to summarize
prompt = "Summarise: '#Person1#: Watsup, ladies! Y'll looking'fine tonight. May I have this dance? #Person2#: He's cute! He looks like Tiger Woods! But, I can't dance. . . #Person1#: It's all good. I'll show you all the right moves. My name's Malik. #Person2#: Nice to meet you. I'm Wen, and this is Nikki. #Person1#: How you feeling', vista? Mind if I take your friend'round the dance floor? #Person2#: She doesn't mind if you don't mind getting your feet stepped on. #Person1#: Right. Cool! Let's go!'"

# Combine examples and prompt
combined_prompt = "\n\n".join(few_shot_examples + [prompt])

# Generate inputs
inputs = tokenizer(combined_prompt, return_tensors="pt", max_length=1024, truncation=True, padding=True)

# Generate outputs
outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)

# Decode and print the summary
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Summary: '#Person1# and #Person2# are going to a dance. #Person1# is going to teach #Person2# how to dance.


## Translations with Few-Shot Learning

In [6]:
# Few-shot examples
few_shot_examples = [
    "Translate this dialogue to French:\n"
    "#Person1#: Excuse me, did you see a set of keys? #Person2#: What kind of keys? "
    "#Person1#: Five keys and a small foot ornament. #Person2#: What a shame! I didn't see them. "
    "#Person1#: Well, can you help me look for it? That's my first time here. #Person2#: Sure. "
    "It's my pleasure. I'd like to help you look for the missing keys. #Person1#: It's very kind of you. "
    "#Person2#: It's not a big deal. Hey, I found them. #Person1#: Oh, thank God! I don't know how to thank you, guys. "
    "#Person2#: You're welcome.\n"
    "French Translation: #Person1#: Excusez-moi, avez-vous vu un trousseau de clés ? #Person2#: Quel genre de clés ? "
    "#Person1#: Cinq clés et un petit ornement pour les pieds. #Person2#: Quel dommage ! Je ne les ai pas vues. "
    "#Person1#: Eh bien, pouvez-vous m'aider à les chercher ? C’est ma première fois ici. #Person2#: Bien sûr. "
    "Avec plaisir. J'aimerais vous aider à les retrouver. #Person1#: C’est très gentil à vous. "
    "#Person2#: Ce n’est pas grand-chose. Hé, je les ai trouvées. #Person1#: Oh, merci mon Dieu ! Je ne sais pas comment vous remercier. "
    "#Person2#: Je vous en prie.",

    "Translate this dialogue to French:\n"
    "#Person1#: This is a good basic computer package. It's got a good CPU, 256 megabytes of RAM, and a DVD player. "
    "#Person2#: Does it come with a modem? #Person1#: Yes, it has a built-in modem. You just plug a phone line into the back of the computer. "
    "#Person2#: How about the monitor? #Person1#: A 15 - inch monitor is included in the deal. "
    "If you want, you can switch it for a 17 - inch monitor, for a little more money. "
    "#Person2#: That's okay. A 15 - inch is good enough. All right, I'll take it.\n"
    "French Translation: #Person1#: C’est un bon ordinateur de base. Il a un bon processeur, 256 mégaoctets de RAM et un lecteur DVD. "
    "#Person2#: Est-ce qu'il a un modem ? #Person1#: Oui, il a un modem intégré. Il suffit de brancher une ligne téléphonique à l’arrière. "
    "#Person2#: Et l’écran ? #Person1#: Un écran de 15 pouces est inclus. Vous pouvez passer à un 17 pouces pour un peu plus. "
    "#Person2#: C’est bon, un 15 pouces me suffit. Très bien, je le prends."
]

# Translation prompt
prompt = (
    "Translate this dialogue to French:\n"
    "#Person1#: Watsup, ladies! Y'll looking'fine tonight. May I have this dance? "
    "#Person2#: He's cute! He looks like Tiger Woods! But, I can't dance... "
    "#Person1#: It's all good. I'll show you all the right moves. My name's Malik. "
    "#Person2#: Nice to meet you. I'm Wen, and this is Nikki. "
    "#Person1#: How you feeling', vista? Mind if I take your friend'round the dance floor? "
    "#Person2#: She doesn't mind if you don't mind getting your feet stepped on. "
    "#Person1#: Right. Cool! Let's go!\n"
    "French Translation:"
)

#  Combine few-shot examples with prompt
combined_prompt = "\n\n".join(few_shot_examples + [prompt])

# Tokenize and generate
inputs = tokenizer(combined_prompt, return_tensors="pt", max_length=1024, truncation=True, padding=True)
outputs = model.generate(inputs["input_ids"], max_length=300, num_beams=5, early_stopping=True)

# Decode and print
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n=== TRANSLATION OUTPUT ===")
print(translation)


=== TRANSLATION OUTPUT ===
#Person1#: Watsup, les filles! Vous ressemblez à Tiger Woods ! Mais je ne peux pas danser ! #Person1#: C'est tout bon, je vais vous montrer tous les mouvements corrects. Mon nom est Malik. #Person2#: Bienvenue à nous. Je suis Wen, et c'est Nikki.
